In [ ]:
pip install git+https://github.com/tensorflow/examples.git

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tqdm
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

In [ ]:
with zipfile.ZipFile("/content/goblin_images.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/new_folder")

In [ ]:
!tar --gunzip --extract --verbose --file=lfw-funneled.tgz

In [ ]:
goblin_dataset = keras.preprocessing.image_dataset_from_directory(
    directory="new_folder", label_mode=None, image_size=(512,512), batch_size=1, shuffle=True
)
people_dataset = keras.preprocessing.image_dataset_from_directory(
    directory="lfw_funneled", label_mode=None, image_size=(512,512), batch_size=1, shuffle=True
)

In [ ]:
Goblin_dataset_size = 750
people_dataset_size = 13233
goblin_train_size = int(0.7 * Goblin_dataset_size)
goblin_test_size = 750 - goblin_train_size
people_train_size = int(0.7 * people_dataset_size)
people_test_size = 13233 - people_train_size
BUFFER_SIZE = 100
BATCH_SIZE = 1
IMG_WIDTH = 512
IMG_HEIGHT = 512

In [ ]:
#the goblin and people train data set should never touch otherwise one ends up with 4 dimensions and the other ends up with 5
#I don't know and I won't pretend to
goblin_train_dataset = goblin_dataset.take(goblin_train_size)
goblin_test_dataset = goblin_dataset.skip(goblin_train_size)

In [ ]:
goblin_train_dataset

In [ ]:
people_train_dataset = people_dataset.take(people_train_size)
people_test_dataset = people_dataset.skip(people_train_size)

In [ ]:
people_train_dataset

In [ ]:
#up to the exact same in cyclic GAN
sample_person = next(iter(people_train_dataset))

In [ ]:
#doesn't work
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

In [ ]:
#shouldn't work
random_crop(next(iter(goblin_dataset))[0])

In [ ]:
# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  #keeping it between 1 and -1
  image = (image / 127.5) - 1
  return image

In [ ]:
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [540, 540],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [ ]:
#returns a jittered image as a process of numbers
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [ ]:
def preprocess_image_test(image):
  image = normalize(image)
  return image

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
#normalize works
#random_crop and random_jitter doesn't work
goblin_train_dataset = goblin_train_dataset.cache().map(
    normalize, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

goblin_test_dataset = goblin_test_dataset.cache().map(
    normalize, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

people_train_dataset = people_train_dataset.map(
    normalize, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

people_test_dataset = people_test_dataset.map(
    normalize, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
sample_goblin = next(iter(goblin_train_dataset))
sample_person = next(iter(people_train_dataset))
sample_goblin